# Jupyter Notebook to use the SPCE method.

This jupyter notebook consideres an example representing a Probability Density Function (PDF), which yields a bimodal distributed output, given as
$$
    f_{Y|X}(y\,|\,x) = w_1\,f_{1,Y|X}\,(y\,|\,x, \mu_1, \sigma_1) + w_2 \,f_{2,Y|X}\,(y\,|\, x, \mu_2, \sigma_2)
$$
which is constructed by the combination of two normal PDF functions $f_{i,Y|X}$, with $i=\{1,\,2\}$, using the weights $w_1 = 0.4$ and $w_2 = 0.6$. The normal PDF is defined as 

$$
    f_{i,Y|X}\,(y\,|\, x, \mu_i, \sigma_i) = \frac{1}{\sigma_i \sqrt{2 \pi}} \cdot \exp{\left(-0.5 \left(\frac{y-\mu_i(x)}{\sigma_i}\right) ^2 \right)}.
$$

Both means $\mu_1$ and $\mu_2$ depend on the input variable $X$ and are defined as
$$
    \begin{split}
    \mu_1(x) &= \sigma_1 \left(5 \sin^2{(\pi x)} + 5x - 2.5 \right),\\
    \mu_2(x) &= \sigma_2 \left(5 \sin^2{(\pi x)} - 5x + 2.5 \right)
    \end{split}
$$

with a standard deviation of 
$$
    \sigma_1 = \sigma_2 = 0.8.
$$

The required packages to run the following code are listed in the file requirements.txt. 

In [14]:
pip install -r requirements.txt

Firstly, load the required libraries.

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import chaospy as cp
from analytical_example_bimodal import AnalyticalExample
from spce import SPCE
import time

Construct the example as follows:

In [2]:
n_samples = 1600
dist_X = cp.Uniform(0, 1)
samples_x = dist_X.sample(size=n_samples, rule='H') 
samples_x_i = np.array([0.2, 0.5, 0.75, 0.9])
indices = [np.abs(samples_x - value).argmin() for value in samples_x_i]
y = np.linspace(-4, 8, 1000)

example = AnalyticalExample(n_samples, y)
pdf, mean_1, mean_2, sigma_1, sigma_2, mean_12, sigma_12 = example.calculate_pdf(samples_x)
samples_y = example.create_data_points(mean_1, mean_2, sigma_1, sigma_2, 1, samples_x).reshape(-1)

Plot the example and the PDFs for specific x-values.

In [12]:
example.plot_example(samples_x, samples_y, mean_1, mean_2, pdf, indices)
example.plot_pdf(pdf, samples_x_i, indices)

Initialzize parameters for constructing the surrogate.

In [4]:
p = 5
dist_Z = cp.Uniform(-1, 1)
dist_joint = cp.J(dist_X, dist_Z)
N_q = 10
q = 0.5

spce = SPCE(n_samples, p, samples_y.T, samples_x, dist_joint, N_q, dist_Z, q)

Compute initial coefficients.

In [5]:
surrogate_q0, poly_initial = spce.start_c(samples_x)
optimized_c = poly_initial.coefficients
polynomials = cp.prod(poly_initial.indeterminants**poly_initial.exponents, axis=-1)

Conduct the warm-up strategy.

In [6]:
error_loo = spce.loo_error(mean_12, surrogate_q0, samples_x)
sigma_range = (0.1 * np.sqrt(error_loo), 1 * np.sqrt(error_loo))
sigma_noise_range = np.linspace(sigma_range[0], sigma_range[-1], 5)
sigma_noise_sorted = sorted(np.exp(sigma_noise_range), reverse=True)

for sigma_noise_i in sigma_noise_sorted:
    optimized_c, message = spce.compute_optimal_c(samples_y, sigma_noise_i, optimized_c, polynomials, samples_x)

c:\Users\carlo\Masterarbeit\Masterarbeit\spce.py:40: RuntimeWarning: divide by zero encountered in log
  likelihood_sum = - np.sum(np.log(likelihood))
c:\Users\carlo\Masterarbeit\Masterarbeit\spce.py:64: RuntimeWarning: divide by zero encountered in divide
  grad = -np.sum((1 / (like)) * (grad_like_sum), axis=1)
c:\Users\carlo\Masterarbeit\Masterarbeit\spce.py:64: RuntimeWarning: invalid value encountered in multiply
  grad = -np.sum((1 / (like)) * (grad_like_sum), axis=1)
c:\Users\carlo\Masterarbeit\Masterarbeit\spce.py:40: RuntimeWarning: divide by zero encountered in log
  likelihood_sum = - np.sum(np.log(likelihood))


Select if the the Maximum Likelihood Estimation (MLE) or the Cross-Validation (CV) method should be used to optimize the coefficients. Comment out the unselected algorithm.

In [7]:
### MLE
for j in range(15):
    sigma_noise = spce.optimize_sigma(samples_y, optimized_c, polynomials, samples_x, sigma_range)
    optimized_c, message = spce.compute_optimal_c(samples_y, sigma_noise, optimized_c, polynomials, samples_x)

### CV 
# sigma_noise = spce.compute_optimal_sigma(optimized_c, polynomials, sigma_range) 
# optimized_c, message = spce.compute_optimal_c(samples_y, sigma_noise, optimized_c, polynomials, samples_x)

Optimized sigma: 1.343900263595698
Optimized sigma: 0.9657150670415634
Optimized sigma: 0.7826079712141227
Optimized sigma: 0.7045450098846529
Optimized sigma: 0.67612228490078
Optimized sigma: 0.6673946680081881
Optimized sigma: 0.664864182983246
Optimized sigma: 0.6641441029747402
Optimized sigma: 0.6639401995401785
Optimized sigma: 0.6638823846467149
Optimized sigma: 0.6638660291006305
Optimized sigma: 0.6638614052163158
Optimized sigma: 0.6638600866878631
Optimized sigma: 0.6638597322350096
Optimized sigma: 0.6638596368485834


Generate test input samples and create output samples of the analytical example.

In [8]:
dist_eps = cp.Normal(0, sigma_noise)
n_x = 1000
n_samples_test = 10000

samples_x_test = dist_X.sample(n_x, rule='H')
samples_z_test = dist_Z.sample(n_samples_test)
samples_eps_test = dist_eps.sample(n_samples_test)

Generate the output distribution predicted by the SPCE surrogate.

In [9]:
dist_spce = spce.generate_dist_spce(samples_z_test, samples_eps_test, optimized_c, polynomials, samples_x_test)

Compute the error using the Wasserstein distance. 

In [10]:
pdf_test, mean_1_test, mean_2_test, sigma_1_test, sigma_2_test, mean_12_test, sigma_12_test = example.calculate_pdf(samples_x_test)
samples_y_test = example.create_data_points(mean_1_test, mean_2_test, sigma_1_test, sigma_2_test, n_samples_test, samples_x_test)

error_spce = spce.compute_error(dist_spce, samples_y_test)
print('Error SPCE: ', error_spce)

Compare the error with that of a Gaussian Process Regression (GPR).

In [11]:
mean_prediction_gpr, std_prediction_gpr, dist_gpr = spce.generate_dist_gpr(samples_x_test, mean_12_test, sigma_12_test)
error_gpr = spce.compute_error(dist_gpr, samples_y_test)
print('Error GPR: ', error_gpr)